Import library, buat koneksi dan function

In [1]:
# Importing packages
import mysql.connector
import pandas as pd

In [2]:
# Creating connection
mydb=mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='******',
    database='world')

mycursor=mydb.cursor()
# Create access to database
query='show tables;'
# Query
mycursor.execute(query)
# Execute query
result=mycursor.fetchall()
# Save result
df=pd.DataFrame(result,columns=mycursor.column_names)
df.head()

,Tables_in_world
0,city
1,country
2,countrylanguage


In [3]:
# Creating Function
mycursor=mydb.cursor()
def sql(query):
    mycursor.execute(query)
    result=mycursor.fetchall()
    df=pd.DataFrame(result,columns=mycursor.column_names)
    return df

In [4]:
sql('SELECT * FROM country LIMIT 5')

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,JosÃ© Eduardo dos Santos,56,AO
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62,AI
4,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,ShqipÃ«ria,Republic,Rexhep Mejdani,34,AL


In [5]:
sql('SELECT * FROM countrylanguage LIMIT 5;')

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3
1,ABW,English,F,9.5
2,ABW,Papiamento,F,76.7
3,ABW,Spanish,F,7.4
4,AFG,Balochi,F,0.9


Tampilkan 10 nama negara (Name) dan jumlah bahasa (language) baik yang official maupun tidak di tiap negara, dari tabel Country dan CountryLanguage! Tampilkan hanya negara yang jumlah bahasanya di atas rata-rata jumlah bahasa tiap negara! Urutkan dari negara yang jumlah bahasanya paling banyak!

,Country_Name,Total_Language
0,United States,12
1,Russian Federation,12
2,India,12
3,China,12
4,Canada,12
5,South Africa,11
6,Tanzania,11
7,Nigeria,10
8,Mozambique,10
9,Iran,10


Permasalahan ada pada AVG(COUNT(Language)): invalid use of group function

In [70]:
sql('''
WITH 
Jumlah_Bahasa as (SELECT CountryCode, Count(Language) as Jumlah_Bahasa FROM CountryLanguage GROUP BY CountryCode),
Mean_Bahasa as (SELECT AVG(Jumlah_Bahasa) as Mean_Bahasa FROM Jumlah_Bahasa)

SELECT Country.Name as Country_Name, Jumlah_Bahasa as Total_Language FROM Country, CountryLanguage, Jumlah_Bahasa
WHERE
Country.Code = CountryLanguage.CountryCode and
Jumlah_Bahasa.CountryCode = Country.Code

GROUP BY Country_Name HAVING Total_Language > (SELECT Mean_Bahasa FROM Mean_Bahasa)
ORDER BY Total_Language DESC LIMIT 10;
''')

,Country_Name,Total_Language
0,Canada,12
1,Russian Federation,12
2,India,12
3,United States,12
4,China,12
5,South Africa,11
6,Tanzania,11
7,Sudan,10
8,Mozambique,10
9,"Congo, The Democratic Republic of the",10


In [76]:
sql('''
WITH Total_Bahasa as
(SELECT Country.Name as Negara, COUNT(CountryLanguage.Language) as Total_Bahasa FROM Country, CountryLanguage
WHERE Country.Code=CountryLanguage.CountryCode
GROUP BY Negara),
Rata_Bahasa as 
(SELECT AVG(Total_Bahasa) as Rata_Bahasa FROM Total_Bahasa)
SELECT * FROM Total_Bahasa
WHERE Total_Bahasa > (SELECT Rata_Bahasa FROM Rata_Bahasa)
ORDER BY Total_Bahasa DESC LIMIT 10;
''')

,Negara,Total_Bahasa
0,United States,12
1,Russian Federation,12
2,India,12
3,China,12
4,Canada,12
5,South Africa,11
6,Tanzania,11
7,Nigeria,10
8,Mozambique,10
9,Iran,10


Tampilkan 10 nama negara (Name), bentuk pemerintahan (GovernmentForm), jumlah population semua negara (Population), persentase populasi tiap negara, serta nomor index baris tiap negara dari tabel Country!

,Name,GovernmentForm,World_Population,Percentage,Row_Index
0,China,People'sRepublic,6078749450,21.0168,42
1,India,Federal Republic,6078749450,16.6755,100
2,United States,Federal Republic,6078749450,4.5792,224
3,Indonesia,Republic,6078749450,3.4893,99
4,Brazil,Federal Republic,6078749450,2.7985,31
5,Pakistan,Republic,6078749450,2.5743,165
6,Russian Federation,Federal Republic,6078749450,2.4172,182
7,Bangladesh,Republic,6078749450,2.1247,22
8,Japan,Constitutional Monarchy,6078749450,2.0845,110
9,Nigeria,Federal Republic,6078749450,1.8344,156


In [107]:
sql('''
SELECT Name, GovernmentForm, SUM(Population) OVER() as World_Population,
Population/SUM(Population) OVER()*100 as Percentage, Row_Number() OVER() as Row_Index
FROM Country
ORDER BY Percentage DESC LIMIT 10;
''') #SUM (Population) OVER() merupakan kesatuan, ga boleh *100 di tengah

,Name,GovernmentForm,World_Population,Percentage,Row_Index
0,China,People'sRepublic,6078749450,21.0168,42
1,India,Federal Republic,6078749450,16.6755,100
2,United States,Federal Republic,6078749450,4.5792,224
3,Indonesia,Republic,6078749450,3.4893,99
4,Brazil,Federal Republic,6078749450,2.7985,31
5,Pakistan,Republic,6078749450,2.5743,165
6,Russian Federation,Federal Republic,6078749450,2.4172,182
7,Bangladesh,Republic,6078749450,2.1247,22
8,Japan,Constitutional Monarchy,6078749450,2.0845,110
9,Nigeria,Federal Republic,6078749450,1.8344,156


Tampilkan jumlah kota di tiap kawasan regional (Region)! Tampilkan hanya kawasan di Benua Asia dan Eropa, serta tambahkan nomor sesuai jumlah kawasan regional di tiap benua (Continent)! Gunakan tabel City dan Country! Perhatikan contoh hasilnya seperti di bawah ini!

In [143]:
sql('''
SELECT Country.Continent as Continent, Country.Region as Region, COUNT(City.Name) as Number_of_City,
ROW_NUMBER() OVER(PARTITION BY Continent) as Row_Group
FROM Country, City
WHERE Country.Code=City.CountryCode and
Country.Continent in ("Asia","Europe")
GROUP BY Region;
''')

,Continent,Region,Number_of_City,Row_Group
0,Asia,Southern and Central Asia,555,1
1,Asia,Middle East,174,2
2,Asia,Southeast Asia,297,3
3,Asia,Eastern Asia,740,4
4,Europe,Southern Europe,156,1
5,Europe,Western Europe,186,2
6,Europe,Eastern Europe,371,3
7,Europe,Nordic Countries,35,4
8,Europe,Baltic Countries,10,5
9,Europe,British Islands,83,6


Kenapa tidak bisa pakai or disini?

In [6]:
sql('''
SELECT Country.Region as Region, COUNT(City.Name) as Number_of_City FROM Country, City
WHERE Country.Code=City.CountryCode and
(Country.Continent="Asia" or Country.Continent="Europe")
GROUP BY REGION;
''')

,Region,Number_of_City
0,Southern and Central Asia,555
1,Southern Europe,156
2,Middle East,174
3,Western Europe,186
4,Eastern Europe,371
5,Southeast Asia,297
6,Eastern Asia,740
7,Nordic Countries,35
8,Baltic Countries,10
9,British Islands,83


Segmen konsumen yang potensial biasanya banyak tinggal di daerah ibukota negara. Meskipun, tidak menutup kemungkinan kota di luar ibukota juga mengalami perkembangan secara pesat, baik secara kuantitas pasar maupun daya beli konsumen.  

GNP suatu negara tidak sedikit yang disumbang perusahaan-perusahaan yang beroperasi di daerah ibukota. Terkadang pabriknya di luar ibukota, namun kantor pusatnya akan ditempatkan di ibukota. Sehingga, warga ibukota merupakan pasar potensial, karena banyak dari mereka yang bekerja di perusahaan-perusahaan besar penyumbang GNP nasional. Maka, GNP kurang lebih juga mencerminkan daya beli warga di ibukota suatu negara.  

Untuk menyusun strategi global, sebuah multinational company membutuhkan data peringkat benua berdasarkan jumlah populasi warga ibukota dan peringkat benua berdasarkan rata-rata GNP semua negara di benua tersebut.  

Tulis query SQL dari tabel yang tersedia database World untuk mendapatkan output tabel seperti di bawah ini! Menurut analisis Anda, benua manakah yang perlu diprioritaskan untuk pemasaran produk perusahaan?  

In [144]:
sql('SELECT * FROM city')

,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200
...,...,...,...,...,...
4074,4075,Khan Yunis,PSE,Khan Yunis,123175
4075,4076,Hebron,PSE,Hebron,119401
4076,4077,Jabaliya,PSE,North Gaza,113901
4077,4078,Nablus,PSE,Nablus,100231


In [145]:
sql('SELECT * FROM country')

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,JosÃ© Eduardo dos Santos,56.0,AO
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
4,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,ShqipÃ«ria,Republic,Rexhep Mejdani,34.0,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,YEM,Yemen,Asia,Middle East,527968.0,1918.0,18112000,59.8,6041.0,5729.0,Al-Yaman,Republic,Ali Abdallah Salih,1780.0,YE
235,YUG,Yugoslavia,Europe,Southern Europe,102173.0,1918.0,10640000,72.4,17000.0,NaN,Jugoslavija,Federal Republic,Vojislav KoÂštunica,1792.0,YU
236,ZAF,South Africa,Africa,Southern Africa,1221037.0,1910.0,40377000,51.1,116729.0,129092.0,South Africa,Republic,Thabo Mbeki,716.0,ZA
237,ZMB,Zambia,Africa,Eastern Africa,752618.0,1964.0,9169000,37.2,3377.0,3922.0,Zambia,Republic,Frederick Chiluba,3162.0,ZM


,Continent,Total_Capital_Population,Average_GNP,Rank_Population,Rank_GNP
0,Asia,101322793,150105.725490,1,3
1,Europe,51917214,206497.065217,2,2
2,Africa,47714282,10182.017544,3,6
3,South America,28900527,107991.000000,4,4
4,North America,19120216,261854.789189,5,1
5,Oceania,1142110,15547.211111,6,5


Ga bisa sembarangan ambil koneksi dari tabel A ke tabel B. Karena hanya mengambil kapital, jadi harus country.capital=city.id;
kalau pakai countrycode akan menampilkan populasi total, bukan hanya ibukota.

In [7]:
sql('''
SELECT Country.Continent as Continent, SUM(City.Population) as Total_Capital_Population, AVG(Country.GNP) as Average_GNP,
RANK() OVER(ORDER BY SUM(City.Population) DESC) as Rank_Population, RANK() OVER(ORDER BY AVG(Country.GNP) DESC) as Rank_GNP
FROM Country, City
WHERE Country.Capital=City.ID
GROUP BY Continent
ORDER BY Rank_Population;
''')

,Continent,Total_Capital_Population,Average_GNP,Rank_Population,Rank_GNP
0,Asia,101322793,150105.725490,1,3
1,Europe,51917214,206497.065217,2,2
2,Africa,47714282,10182.017544,3,6
3,South America,28900527,107991.000000,4,4
4,North America,19120216,261854.789189,5,1
5,Oceania,1142110,15547.211111,6,5


1. Paragraf di atas hanya bersifat spekulatif dan tidak berdasar pada data. Data GNP itu bersifat nasional, bukan ibukota saja. Mungkin pernyataan: ibukota menjadi penyumbang GNP terbesar bisa saja benar, tapi apakah dia menyumbangkan hingga 95% dari keseluruhan nilai GNP? Tentu saja tidak, kecuali negaranya hanya ada 2 kota, ibukota dan 1 kota tertinggal. Lalu, rumus GNP adalah pendapatan WN di dalam negeri + pendapatan WN di luar negeri, sehingga jelas bahwa tidak ada aspek lokasi untuk nilai keseluruhan GNP. Jadi, poin pertama adalah bias jika menggunakan total populasi ibukota sebagai parameter pengambilan keputusan untuk penjualan/ekspansi karena tidak menggambarkan daya beli dari masyarakat (yang mana menjadi tujuan perusahaan).

2. Daya beli masyarakat tentunya sudah jelas dipengaruhi dari nilai GNP. Sehingga, jika ingin memprioritaskan kepada kemampuan berbelanja masyarakat, GNP sudah menjadi tolak ukur utama untuk ekspansi. Walaupun bisa jadi penerimaan WN di luar negeri bisa berbeda kontinen. Tapi harusnya kecil dibandingkankan di dalam negeri.

3. Tapi bagaimana jika memang bos memaksakan kehendak untuk juga mempertimbangkan aspek total populasi ibukota? Jika memang kedua hal itu penting?
Jawabannya:

In [163]:
sql('''
SELECT Country.Continent as Continent, SUM(City.Population) as Total_Capital_Population, AVG(Country.GNP) as Average_GNP,
AVG(Country.GNP)/SUM(City.Population) OVER() as Ratio
FROM Country, City
WHERE Country.Capital=City.ID
GROUP BY Continent
ORDER BY Ratio;
''')

,Continent,Total_Capital_Population,Average_GNP,Ratio
0,Africa,47714282,10182.017544,0.001437
1,Oceania,1142110,15547.211111,0.002194
2,South America,28900527,107991.000000,0.015241
3,Asia,101322793,150105.725490,0.021185
4,Europe,51917214,206497.065217,0.029144
5,North America,19120216,261854.789189,0.036956


4. Tetap saja North America menjadi prioritas utama, jika membandingkan GNP dengan total populasi di kapital, walaupun dari poin 1 udah bias jika dilakukan. GDP mungkin bisa menjadi alternatif lain, dengan membandingkan total GDP/total topulasi keseluruhan di suatu negara atau rata-rata GDP kontinen/total populasi kontinen.

Sebuah multinational company lain memiliki pendekatan strategi yang lain. Bukan dengan pendekatan benua prioritas target pasar, melainkan langsung menyasar negara prioritas. Perusahaan ini juga tidak menekankan kuantitas pasar (market size), tapi lebih ke daya beli pasar (consumer's buying power).  
Salah satu aspek yang mencerminkan daya beli pasar di suatu negara adalah GNP. Asumsinya semakin tinggi GNP suatu negara, maka semakin prioritas menjadi target product marketing. Dengan hanya menyasar negara-negara yang menyumbang 80 persen dari total GNP Global, maka operasi pemasaran akan lebih efisien.  
Perusahaan menugaskan Anda untuk mendapatkan output table seperti di bawah ini dari World Database. Tulis SQL query untuk menampilkan:  
* Nama negara prioritas,
* Persentase GNP tiap negara dibandingkan total GNP Global,
* Nilai kumulatif (Moving Sum) dari persentase GNP hanya sampai 80 persen saja,
* Peringkat berdasarkan persentase GNP, dan
* Kelompokkan negara-negara prioritas ini ke dalam 4 kelompok Market Priority!
* Tampilkan hanya negara-negara yang GNP-nya di atas satu persen!

,Country_Name,GNP_Percentage,Cumulative_GNP,GNP_Rank,Market_Priority_1234
0,United States,28.99,28.99,1,1
1,Japan,12.90,41.89,2,1
2,Germany,7.27,49.16,3,1
3,France,4.85,54.01,4,1
4,United Kingdom,4.70,58.71,5,2
5,Italy,3.96,62.67,6,2
6,China,3.35,66.02,7,2
7,Brazil,2.65,68.67,8,2
8,Canada,2.04,70.71,9,3
9,Spain,1.88,72.59,10,3


In [10]:
sql('''
WITH GNP_Percentage as
(SELECT Name as Negara, GNP/SUM(GNP) OVER()*100 as GNP_Percentage FROM Country 
ORDER BY GNP_Percentage DESC),

GNP_Cumulative as
(SELECT Negara, GNP_Percentage, SUM(GNP_Percentage) OVER(ORDER BY GNP_Percentage DESC 
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as Cumulative_GNP
FROM GNP_Percentage)

SELECT *, 
RANK() OVER(ORDER BY GNP_Percentage DESC) as GNP_Rank, 
NTILE(4) OVER() as Market_Priority_1234 FROM GNP_Cumulative
WHERE Cumulative_GNP<81;
''')

,Negara,GNP_Percentage,Cumulative_GNP,GNP_Rank,Market_Priority_1234
0,United States,28.992426,28.992426,1,1
1,Japan,12.900882,41.893308,2,1
2,Germany,7.267497,49.160805,3,1
3,France,4.851948,54.012753,4,1
4,United Kingdom,4.695399,58.708152,5,2
5,Italy,3.957618,62.665770,6,2
6,China,3.346180,66.011950,7,2
7,Brazil,2.646028,68.657977,8,2
8,Canada,2.040075,70.698052,9,3
9,Spain,1.884635,72.582687,10,3
